# Dynamic programming and memoization

In class we looked at the Levenshtein distance between two strings and how it can be computed efficiently with dynamic programming.
Dynamic programming comprises two key techniques:

1. Break down large problems into small problems.
1. Store the solutions to small problems so that they do not have to be recomputed (**memoization**).

All of you have actually used dynamic programming before.
In grade school, you probably had to solve equations like `5 + (3 * 4) - (4 * (2 + 1))`.
After a few exercises like that, you realized that it's easier if you don't just try to solve that in one fell swoop.
Instead, you broke it down into smaller problems:

- `3 * 4` is `12`.
- `2 + 1` is `3`.
- `4 * 3` is the same as `3 * 4`, so you can just reuse the `12` you've computed before.
- `12 - 12` is `0`.
- `5 + 0` is `5`.

This unit will cover how dynamic programming can be used in Python with dictionaries.
The first notebook (i.e. this one) will introduce recursive functions as an example of where memoization can greatly improve your code's efficiency.
The second notebook then explains how memoization works, and the last one applies these techniques to the Levenshtein distance.

## Background: Computing the factorial

The **factorial** `n!` of a positive integer `n` is the result of multiplying `n` with the factorial of `n-1`.
The exception to this rule is `1`, its factorial is just `1`.
With numbers less than `1`, the factorial is usually undefined, but for simplicity we will assume that it is also `1`.
A few examples:

- `5! = 5 * 4! = 5 * 4 * 3! = 5 * 4 * 3 * 2! = 5 * 4 * 3 * 2 * 1! = 5 * 4 * 3 * 2 * 1= 120`
- `-5! = 1`.
- `1! = 1`.

The factorial grows very, very fast.
To see how fast, we can write a Python function for computing the factorial.
It's actually very easy as we can just use `range`.

In [ ]:
def factorial(n):
    """Return the factorial of n."""
    # for numbers less than or equal to 1, just return 1
    if n <= 1:
        return 1
    else:
        for i in range(n):
            if i > 0:
                n *= i
        return n

In [ ]:
for i in range(10):
    print(f"{i}! = {factorial(i)}")

In [ ]:
# a really large factorial
print(f"{10000}! = {factorial(10000)}")

The factorial grows faster than any exponential.
That is to say, even a mindboggling function like `f(x) = 10,000,000^x` will return smaller numbers than the factorial once one reaches large values for `x`.

A brief remark one the code above.
The `if > 0` condition is used to exclude `0` from the values in `range(n)`.
After all, we do not want to multiply by `0` since that would give us `0`.
But the `if` is still a little clunky.
Instead, we can tell `range` directly to exclude it.
By passing `range(1, n)`, we instruct `range` to start at `1` instead of `0`.

In [ ]:
print(list(range(1, 5)))

Quite generally, `range(x, y)` means "start counting from `x` and go up to (but excluding) `y`".

In [ ]:
print(list(range(-5, 5)))

As you can see `range(x)` is just a shorthand for `range(0, x)`.

Anyways, back to the factorial.
It is a function with ludicrously fast growth, and we can implement it very easily in Python with `range` and multiplication.
But our function is actually a bit different from the definition we saw at the beginning of this section.
There, it said that the factorial of `n` is `n` multiplied with the factorial of `n-1`.
Let's try to translate this in a literal fashion into Python code.

In [ ]:
def factorial(n):
    """Return the factorial of n."""
    # base case: factorial of 1 and lower
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

This looks a bit peculiar.
A return value of `n * factorial(n-1)`, will that actually work?
Can a function call itself?
Yes, it actually can, and the code actually work.

In [ ]:
for i in range(10):
    print(f"{i}! = {factorial(i)}")

In [ ]:
# don't worry about sys.setrecursionlimit for now,
# we'll discuss it at the end of the next notebok
import sys
sys.setrecursionlimit(10000)

# a really large factorial
print(f"{10000}! = {factorial(10000)}")

We've just encountered our first **recursive function**.

## Recursive functions: elegance + inefficiency

### Factorial with recursion

A recursive function is a function that calls itself during the computation.
Just look at how the recursive implementation of the factorial computes `factorial (5)`:

1. `factorial(5)` is `5 * a`. To determine `a`, we have to calculate `factorial(4)`.
1. `factorial(4)` is `4 * b`, where `b` is whatever is returned by `factorial(3)`.
1. `factorial(3)` is `3 * c`, where `c` is the output of `factorial(2)`.
1. `factorial(2)` is `2 * d`, and now `d` is `factorial(1)`.
1. `factorial(1)` returns `1`.
1. `factorial(2)` then returns `2 * 1`, i.e. `2`.
1. `factorial(3)` next returns `3 * 2`, which is `6`.
1. `factorial(4)` then returns `24`, the result of `4 * 6`.
1. `factorial(5)` finally computes the final result: `5 * 24`, which is `120`.

We can represent this process as a diagram of nested function calls:

```
factorial(5)
  |
5 * factorial(4)
      |
    4 * factorial(3)
          |
        3 * factorial(2)
            |
          2 * factorial(1)
                  |
                  1
```

As you can see here, we keep drilling deeper and deeper into this structure until we finally find a case where `factorial` does not require computing another instance of `factorial`.
This is the **base case**.
In our code, the base case is established by the condition `if n <= 1`.
If we removed this part, `factorial` would just keep calling itself without ever returning a value.
It's the same like a `while`-loop that never stops.

To see this, run the cell below with a broken version of `factorial` that lacks the base case.
For safety reasons, the code limits Python to 100 recursion steps.
Even computing the factorial of `2` will hit this limit because the function just keeps calling itself with lower and lower integers (1, 0, -1, -2, ...) but never hits a base case.

In [ ]:
# limit Python to 100 recursion steps
import sys
sys.setrecursionlimit(100)

def broken_factorial(n):
    """Return the factorial of n."""
    return n * broken_factorial(n-1)

broken_factorial(2)

As you can see, you always need a base step or you will just keep recursing forever.
Remember, a recursive function always needs two parts:

1. a **base step** that returns the value for the simplest case,
1. a **recursion step** that computes the value of a complex case in terms of a simpler case.

Recursive functions can be very elegant, in particular for mathematical problems.
If you open up a mathematics textbook, you will come across many defintions that exhibit a similar kind of recursion.
This makes recursive functions a very natural way of implementation.
But recursive functions can also be very inefficient, as we'll see next.

### Inefficient recursion: Fibonacci numbers

The Fibonacci series is a sequence of numbers of the following form:

```
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...
```

This is an intuitively pleasing pattern.
We start with `1` and `1`, and then each number is the sum of the previous two.
The Fibonacci series is related to the [Golden ratio](https://en.wikipedia.org/wiki/Golden_ratio) and shows up a lot in nature.
So it is an interesting series, and it would be nice to have a way to compute the `n`-th Fibonacci number.

Since the definition of the Fibonacci series is recursive, a recursive function seems like a natural candidate.

In [ ]:
def fibonacci(n):
    """Recursive implementation of Fibonacci series.
    
    Only defined for n >= 1."""
    # base case for the first two numbers
    if n == 1 or n == 2:
        return 1
    # recursion step
    else:
        return fibonacci(n-1) + fibonacci(n-2)

This is the literal Python counterpart to the mathematical definition above.
But it is also really, really, really slow.
Even computing the 50th Fibonacci number will take forever.
Try it yourself: run the cell below, and go fix yourself a coffee.
I'm not joking, take a break, relax, come back in 20 minutes, and perhaps it will be done by then.

In [ ]:
print(fibonacci(50))

Why is this so slow while the recursive implementation of `factorial` didn't even choke on `factorial(10000)`?
The answer lies in the recursion used by `fibonacci` and how it differs from `factorial`.
Look at the diagram below for `fibonacci(5)`.

```
fibonacci(5)
     |
fibonacci(3) + fibonacci(4)
     |              |
     |         fibonacci(3) + fibonacci(2)
     |              |              |
     |              |              1
     |              |
     |         fibonacci(2) + fibonacci(1)
     |              |              |
     |              1              1
     |
     |
fibonacci(2) + fibonacci(1)
     |              |
     1              1
```

Notice how we keep recomputing the same value over and over again?
For instance, `fibonacci(3)` is computed for both `fibonacci(5)` and `fibonacci(4)`.
We can make a table to list how often each Fibonacci number is computed when calling `fibonacci(5)`.

| Fibonacci number | times computed |
| :-               | :-             |
| 5                | 1              |
| 4                | 1              |
| 3                | 2              |
| 2                | 3              |
| 1                | 2              |

You might think that's not a big deal.
After all, `fibonacci(3)` is just `1 + 1`.
But the problem is that the times we recompute numbers blows up astronomically as we try to compute larger Fibonacci numbers.
Here's the corresponding table for `fibonacci(7)`.

| Fibonacci number | times computed |
| :-               | :-             |
| 7                | 1              |
| 6                | 1              |
| 5                | 2              |
| 4                | 3              |
| 3                | 5              |
| 2                | 8              |
| 1                | 5              |

Do you notice something?
If we ignore the value for 1, then the second column by itself is an instance of the Fibonacci series!
in order to compute the n-th Fibonacci number, the number of times we have to compute the (n-m)-th Fibonacci number is the (m+1)-th Fibonacci number.
For instance, if we're looking for the 7th Fibonacci number, then the value for 3 is computed 5 times, and the 5th Fibonacci number (5 = 7 - 3 + 1) is 5.
This can't be good, right?

Quite right.
To show you how bad this is, the table below lists how often the first 7 numbers are recomputed for `fibonacci(50)`.

| Fibonacci number | times computed |
| :-               | :-             |
| 1                | 4,807,526,976  |
| 2                | 7,778,742,049  |
| 3                | 4,807,526,976  |
| 4                | 2,971,215,073  |
| 5                | 1,836,311,903  |
| 6                | 1,134,903,170  |
| 7                |   701,408,733  |

The total number of computations to get the 50th Fibonacci number with our recursive function is truly mind-boggling: 25,172,538,049.
That's 25 billion, 172 million, 538 thousand and 49 computations.
Just for the 50th Fibonacci number.
No wonder this is taking forever!

But keep in mind that this isn't the intrinsic difficulty of computing the 50th Fibonacci number.
It's just that our solution with a recursive function is a really bad way of doing it.
We keep recomputing the same things over and over again, and that makes the function incredibly slow.

If we had a way not to constantly recompute results, it should only take a split second.
That's exactly what **memoization** is all about, and the next notebook explains in detail how it works.

## Bullet point summary

- A **recursive function** is a function that may call itself during the computation.
- Recursive functions can be very elegant, but this elegance can hide tremendous inefficiency.